In [2]:
!cat ~/mapping_script/mapping_script.txt

#!/bin/bash
#
#SBATCH --job-name=bwa_mem_hg38_samtools_fastq_to_bam_230723
#SBATCH --output=/home/ndo/slurm_script/slurm_out_err/bwa_mem_%j.out
#SBATCH --error=/home/ndo/slurm_script/slurm_out_err/bwa_mem_%j.err
#
#SBATCH --ntasks=2
#SBATCH --cpus-per-task=28
#SBATCH --mem-per-cpu=4G

datenow=$(date)
echo $datenow
echo ""
srun hostname
echo ""

start=$(date +%s)
echo "start time: $start"
echo ""
echo "hostname: $HOSTNAME"
echo ""

##################

#eval "$(conda shell.bash hook)"
#conda activate conda_fastqc                      ### CONDA ENVIRONMENT NAME
#conda info
#echo ""

module load bwa
module load samtools

call="bwa" # to get the version
echo "bwa version"
echo $call
eval $call
echo ""

fastq1=$1                             ### INPUT FASTQ1
fastq2=$2                             ### INPUT FASTQ2
output_dir=$3                         ### OUTPUT DIRECTORY 

echo "output directory: $output_dir"
echo ""

out_name=$(basename $fastq1 _1.fastq)
echo $out_name

echo $output_dir$out_n

In [4]:
# submit mapping job to SLUMR HPC

!sbatch mapping_script/mapping_script.txt \
    ./00_raw_fastq/SRR13586007_1.fastq \
    ./00_raw_fastq/SRR13586007_2.fastq \
    ./01_raw_bam/

!sleep 5
!squeue -u ndo
!sleep 10
!squeue -u ndo
!sleep 15
!squeue -u ndo
!sleep 30
!squeue -u ndo

Submitted batch job 1108860
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1108858     nodes     bash      ndo  R       7:11      1 node01
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1108858     nodes     bash      ndo  R       7:22      1 node01
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1108858     nodes     bash      ndo  R       7:37      1 node01
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1108858     nodes     bash      ndo  R       8:07      1 node01


In [8]:
!squeue -u ndo

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1108708     nodes     bash      ndo  R    2:00:36      1 node01
           1108713     nodes bwa_mem_      ndo  R       0:48      2 node[08-09]


In [5]:
!squeue -u ndo

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1108708     nodes     bash      ndo  R      36:00      1 node01
           1108709     nodes bwa_mem_      ndo  R      28:45      2 node[08-09]


In [1]:
!conda deactivate



CommandNotFoundError: Your shell has not been properly configured to use 'conda deactivate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [25]:
!cat /home/ndo/picard_sort/sort_script.txt

#!/bin/bash
##
#SBATCH --job-name=picard_sort
#SBATCH --output=/home/ndo/slurm_script/slurm_out_err/sorted_bam_%j.out
#SBATCH --error=/home/ndo/slurm_script/slurm_out_err/sorted_bam_%j.err
##
#SBATCH --ntasks=2
#SBATCH --cpus-per-task=28
#SBATCH --mem-per-cpu=4G

bam_file="$1"     #INPUT AS A BAM FILE
input_dir="/home/ndo/01_raw_bam/"
out_dir="/home/ndo/02_sorted_bam/"

out_name="$(realpath "${out_dir}/${bam_file}")"
in_name="$(realpath "${input_dir}/${bam_file}")"
echo "outname"
realpath "${out_dir}/${bam_file}"
echo "inname"
realpath "${input_dir}/${bam_file}"

gatk

call=$(gatk SortSam \
        --INPUT $in_name \
        --OUTPUT $out_name \
        --VERBOSITY DEBUG \
        --TMP_DIR /gpfs/scratch/ndo \
        --SORT_ORDER queryname)

echo $call
eval $call

echo "finish sorting bam file"


In [31]:
!sbatch /home/ndo/picard_sort/sort_script.txt \
        SRR13586007.bam

Submitted batch job 1112929


In [32]:
!squeue -u ndo

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1112929     nodes picard_s      ndo  R       0:03      2 node[20-21]


In [39]:
!cat /home/ndo/removedup_script/removedup_script.txt

#!/bin/bash
#
#SBATCH --job-name=remove_dup
#SBATCH --output=/home/ndo/slurm_script/remove_dup%j.out
#SBATCH --error=/home/ndo/slurm_script/remove_dup%j.err
#
#SBATCH --ntasks=2
#SBATCH --cpus-per-task=28
#SBATCH --mem-per-cpu=4G

bam_file="$1"     #INPUT AS A BAM FILE
input_dir="/home/ndo/02_sorted_bam/"
out_dir="/home/ndo/03_removedup_bam/"

out_name="$(realpath "${out_dir}/${bam_file}")"
in_name="$(realpath "${input_dir}/${bam_file}")"
echo "outname"
realpath "${out_dir}/${bam_file}
echo ""
echo "inname"
realpath "${input_dir}/${bam_file}"

# try the memmory issue
call=$(gatk Mark\
      -I $in_name \
      -O $out_name \
      -M /home/ndo/removedup_script/marked_dup_metrics.txt \
      -REMOVE_DUPLICATES true
      --VERBOSITY DEBUG)

echo $call
eval $call

echo "finish remove dup from bam file"

In [43]:
!sbatch /home/ndo/removedup_script/removedup_script.txt \
        SRR13586007.bam

!sleep 5
!squeue -u ndo
!sleep 10
!squeue -u ndo
!sleep 15
!squeue -u ndo
!sleep 30
!squeue -u ndo

Submitted batch job 1112993


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1112993     nodes remove_d      ndo  R       0:05      2 node[20-21]
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1112993     nodes remove_d      ndo  R       0:15      2 node[20-21]
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1112993     nodes remove_d      ndo  R       0:30      2 node[20-21]
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1112993     nodes remove_d      ndo  R       1:01      2 node[20-21]


In [44]:
!squeue -u ndo


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1112993     nodes remove_d      ndo  R       1:28      2 node[20-21]


SyntaxError: invalid syntax (<ipython-input-8-fda4d146add7>, line 1)

In [1]:
conda activate gatk



CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [3]:
!source ~/anaconda3/bin/activate


/bin/bash: /home/ndo/anaconda3/bin/activate: No such file or directory


In [4]:
conda env list

# conda environments:
#
conda_fastqc          *  /home/ndo/.conda/envs/conda_fastqc
gatk                     /home/ndo/.conda/envs/gatk
test1                    /home/ndo/.conda/envs/test1
base                     /opt/intel/intelpython3
RNAseq_R_tools           /opt/intel/intelpython3/envs/RNAseq_R_tools
RNAseq_v1.0              /opt/intel/intelpython3/envs/RNAseq_v1.0
RNAseq_v2.0              /opt/intel/intelpython3/envs/RNAseq_v2.0
_galaxy_                 /opt/intel/intelpython3/envs/_galaxy_
fan-c                    /opt/intel/intelpython3/envs/fan-c
hap.py                   /opt/intel/intelpython3/envs/hap.py
jupyterhub-env           /opt/intel/intelpython3/envs/jupyterhub-env
jupyterhub-env-clone     /opt/intel/intelpython3/envs/jupyterhub-env-clone
ngstools-env             /opt/intel/intelpython3/envs/ngstools-env
trinityRNAseq            /opt/intel/intelpython3/envs/trinityRNAseq


Note: you may need to restart the kernel to use updated packages.


In [5]:
conda init bash


no change     /opt/intel/intelpython3/condabin/conda
no change     /opt/intel/intelpython3/bin/conda
no change     /opt/intel/intelpython3/bin/conda-env
no change     /opt/intel/intelpython3/bin/activate
no change     /opt/intel/intelpython3/bin/deactivate
no change     /opt/intel/intelpython3/etc/profile.d/conda.sh
no change     /opt/intel/intelpython3/etc/fish/conf.d/conda.fish
no change     /opt/intel/intelpython3/shell/condabin/Conda.psm1
no change     /opt/intel/intelpython3/shell/condabin/conda-hook.ps1
no change     /opt/intel/intelpython3/lib/python3.6/site-packages/xontrib/conda.xsh
no change     /opt/intel/intelpython3/etc/profile.d/conda.csh
no change     /home/ndo/.bashrc
No action taken.

Note: you may need to restart the kernel to use updated packages.


In [1]:
conda activate gatk


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [2]:
conda init

no change     /opt/intel/intelpython3/condabin/conda
no change     /opt/intel/intelpython3/bin/conda
no change     /opt/intel/intelpython3/bin/conda-env
no change     /opt/intel/intelpython3/bin/activate
no change     /opt/intel/intelpython3/bin/deactivate
no change     /opt/intel/intelpython3/etc/profile.d/conda.sh
no change     /opt/intel/intelpython3/etc/fish/conf.d/conda.fish
no change     /opt/intel/intelpython3/shell/condabin/Conda.psm1
no change     /opt/intel/intelpython3/shell/condabin/conda-hook.ps1
no change     /opt/intel/intelpython3/lib/python3.6/site-packages/xontrib/conda.xsh
no change     /opt/intel/intelpython3/etc/profile.d/conda.csh
no change     /home/ndo/.bashrc
No action taken.

Note: you may need to restart the kernel to use updated packages.
